In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct")

In [2]:
from langchain_community.vectorstores import Chroma

In [4]:
persist_directory = "../database/anwesha_chroma"
vectorstore = Chroma(
    persist_directory=persist_directory, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

In [6]:
from langchain import hub

In [7]:
prompt = hub.pull("rlm/rag-prompt")

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
rag_chain.invoke("অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?")

'প্রশ্ন ১০-এর উত্তর অনুযায়ী, অনুপম তার মামার চেয়ে **বছর ছয়েক** ছোট ছিল।'

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatGroq(model="moonshotai/kimi-k2-instruct",)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [15]:
from langchain.load import dumps, loads


def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


question = "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

5

In [16]:
docs

[Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='১৪। মামা কেন মেয়ের পক্ষে পক্ষে করতেন?  \n(ক) ধনী (খ) গরিব (গ) প্রমীণ (ঘ) শখলে  \n\n১৫। অনুপমের বন্ধুর নাম কি?  \n(ক) সজল (খ) জ্যোতিষ (গ) হরিশ (ঘ) মণীষ  \n\n১৬। মেয়ের চেয়ে মামার বয়সে খুরবচাই ফার কাছ গুরত্ব?  \n(ক) হরিশের (খ) অনুপমের (গ) মামার (ঘ) ঘটকের'),
 Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='i. মামাই অনুপমের অভিভাবক  \nii. তিনি অনুপমের চেয়ে বড়ভাইর বছর ছয়েকের বড়  \niii. ফজলুল বালিশ মতে তিনি অনুপমের সংসার আয়কড়ে আছেন  \n\nনিচের কোনটি সঠিক?  \n(ক) i, ii  \n(খ) i, iii  \n(গ) ii, iii  \n(ঘ) i, ii, iii  \n\n২২। মামার পছন্দের মেয়ের এমন-'),
 Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='৪৫। মাটু-বালকে কল্যাণী কার প্রতি ঈর্ষিত করেছে?\n\n(ক) মায়ের প্রতি     (খ) মাধুরীর প্রতি     (গ) ধনীর প্রতি     (ঘ) অনুপমের প্রতি\n\n৪৬। বিনুর অনুপমের পুত্রের বয়স কত ছিল?\n\n(ক) ২০ বছর     (খ) ২২ বছর     (গ) ২৫ বছর     (ঘ) ২৭ বছর\n\n৪৭। অনুপমের 

In [18]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'উত্তর: বছর ছয়েক।'

In [19]:
sample_queries = [
    "অপরিচিতা' গল্পে, অনুপমের মতে কে আসর জমাতে অদ্বিতীয়?",
    "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?",
    "মন্দ নয় হে! খাঁটি সোনা বটে!' - এই উক্তিটি কার?",
    "কল্যাণীর বাবার নাম কী?",
    "বিবাহ-উপলক্ষ্যে কন্যাপক্ষকে কোথায় আসতে হয়েছিল?",
    "শম্ভুনাথ সেন পেশায় কী ছিলেন?",
    "অনুপম এবং তার মা কোন বাহনে তীর্থে যাচ্ছিলেন?",
    "রেলগাড়িতে কল্যাণীর সাথে কয়টি ছোট ছোট মেয়ে ছিল?",
    "বিবাহ ভাঙার পর কল্যাণী কী ব্রত গ্রহণ করে?",
    "গল্পের শেষে অনুপমের বয়স কত?"
]

expected_responses = [
    "হরিশ",
    "বছর ছয়েক",
    "বিনুদা",
    "শম্ভুনাথ সেন",
    "কলিকাতা",
    "ডাক্তার",
    "রেলগাড়ি",
    "দুটি-তিনটি",
    "মেয়েদের শিক্ষার ব্রত",
    "সাতাশ"
]

In [24]:
from ragas import EvaluationDataset


dataset = []

for query, reference in zip(sample_queries, expected_responses):
    relevant_docs = retriever.invoke(query)
    response = final_rag_chain.invoke({"question": query})
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

result